# Deploy ChatGLM-v2 on SageMaker

本节实验会在 SageMaker 上使用 Large Model Inference（LMI）镜像对 ChatGLM2-6B 模型进行部署，该镜像包含了以下特性：

- 通过 DJL Serving（基于 java 开发的高性能后端框架）暴露 HTTP 模型终端节点
- 包含多种大模型推理优化引擎（DeepSpeed inference, FasterTransformer, Hugging Face Accelerate），从而降低延迟，提升吞吐
- Tensor Parallel，支持将大模型拆分到多张 GPU 卡上实现推理过程的模型并行
- 支持模型量化与 Attention 层计算优化
- 推理支持流式输出
- 内置 s5cmd 高性能 S3 传输工具，从而实现大模型文件的快速下载

#### Install and upgrade dependencies

In [ ]:
%pip install --upgrade pip --quiet
%pip install sagemaker boto3 awscli --upgrade --quiet

In [ ]:
# init sagemaker parameters
import boto3
import sagemaker
from sagemaker import Model, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_code_prefix = "east-ai-models/chatglm2/accelerate"

In [ ]:
!mkdir mymodel

#### Writing SageMaker LMI code properties and model.py

使用 LMI 镜像需要提供 serving.properties 文件，在里面定义好相关参数配置，其中：
1. engine 为推理优化引擎，本节实验使用 Hugging Face Accelerate 作为推理引擎，将该参数设置为 Python 即可
2. tensor_parallel_degree 决定了张量并行度，本实验只使用 1 张 a10 GPU，因此将该参数设置为 1；若有 2 张 GPU 卡，可以将该参数设置为 2；4 张 GPU 卡，可设置为 4，以此类推
3. model_id 为 Hugging Face 上对应的模型名称，部署时，SageMaker 会从 Hugging Face 下载对应的模型到本地容器环境中
4. enable_streaming 为启用推理流式 token 输出 
对于其他参数的完整解释，可以参考文档：https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html

In [ ]:
%%writefile ./mymodel/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.model_id=THUDM/chatglm2-6b

In [ ]:
%%writefile ./mymodel/requirements.txt
transformers==4.30.2

model.py 为推理时的入口函数，我们只需要编写其中的两个回调函数：1） get_model() 函数控制了加载模型的过程；2） handel() 函数来实现模型的推理，以及对输入输出数据格式的转化。

In [ ]:
%%writefile ./mymodel/model.py
from djl_python import Input, Output
import os
import torch
from transformers import AutoTokenizer, AutoModel
from typing import Any, Dict, Tuple
import warnings
import json

model = None
tokenizer = None


def get_model(properties):
    model_name = properties["model_id"]
    local_rank = int(os.getenv("LOCAL_RANK", "0"))
    model = AutoModel.from_pretrained(model_name, trust_remote_code=True).half().cuda()
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    return model, tokenizer


def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer
    size = 0
    response = ""
    for response, history in model.stream_chat(tokenizer, prompt, history=history, max_length=max_length, top_p=top_p,
                                               temperature=temperature):
        this_response = response[size:]
        history = [list(h) for h in history]
        size = len(response)
        stream_buffer = { "outputs":this_response, "history":history}
        yield stream_buffer


def handle(inputs: Input) -> None:
    global model, tokenizer
    print("print inputs: " + str(inputs) + '.'*20)
    if not model:
        model, tokenizer = get_model(inputs.get_properties())

    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        return None
    input_map = inputs.get_as_json()
    data = input_map.pop("inputs", input_map)
    params = input_map.pop("parameters", {})
    history = input_map["history"]
    print("print data: " + str(data) + '.'*20)
    model = model.eval()
    outputs = Output()
    outputs.add_property("content-type", "application/jsonlines")
    outputs.add_stream_content(stream_items(data, history=history, **params))
    print("Start streaming output" + "."*20)
    return outputs

In [ ]:
# compress code and upload to S3
!rm -f model.tar.gz
!rm -rf mymodel/.ipynb_checkpoints
!tar czvf model.tar.gz -C mymodel .
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar uploaded to --- > {s3_code_artifact}")

#### Model deployment

In [ ]:
# retrieve SageMaker LMI container image URI
image_uri = sagemaker.image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.23.0"
)


print(image_uri)

model = Model(image_uri=image_uri, model_data=s3_code_artifact, role=role)

In [ ]:
instance_type = "ml.g5.2xlarge"  # "ml.g5.2xlarge" - #single GPU. really need one GPU for this since tensor split is '1'

endpoint_name = "chatglm2-lmi-model"

model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=900,
)

#### Prediction

在模型部署完成后，应用端需要定义流式处理的类，来获取模型服务器流式返回的 token，如下所示：

In [ ]:
import io

class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

通过 boto3 调用 invoke_endpoint_with_response_stream 方法请求模型服务器终端节点进行流式推理

In [ ]:
import boto3
import json

smr = boto3.client('sagemaker-runtime')

parameters = {
  "max_length": 4092,
  "temperature": 0.01,
  "top_p":0.8
}

response_model = smr.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": """推荐几个适合度假的地方""",
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
scanner = StreamScanner()
for event in event_stream:
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)['outputs']['outputs']
            print(resp, end='')
            # print(resp.get("outputs")['outputs'], end='')
        except Exception as e:
            # print(line)
            continue